# Clize
## A Python approach to command-line argument parsing

**Yann Kaiser**

Full-time student from France!

[@YannKsr](https://twitter.com/YannKsr)

* Yann Kaiser
* How I decided to approach arg parsing

## What's a CLI?
* In the scope of this talk: The way you start programs from the shell
* A textual command that takes arguments:  
      command argument --option value
      git commit -v -m "Fixed bug #42"

We want to make one! Let's start by writing a function...

* when using a terminal or ssh, the way you start prog from bash, zsh, etc
* textual, takes arguments
* let's make one, specify behavior in a function first

## Hello world!

In [1]:
def greet(name=None, *, no_capitalize=False):
    """Returns a greeting for a given person or the whole world."""
    if name is None:
        return "Hello world!"
    else:
        if not no_capitalize:
            name = name.title()
        return "Hello {}!".format(name)

In [2]:
greet()

'Hello world!'

In [3]:
greet("john")

'Hello John!'

In [4]:
greet("dave", no_capitalize=True)

'Hello dave!'

* Simple function:
  * ``Name`` positional,
  * ``no_capitalize`` keyword
* Also available in Python 2

Next: make a CLI / argparse

## This is how you write a command-line interface for it

In [5]:
import argparse

In [6]:
def greetcli(args=None):
    parser = argparse.ArgumentParser(prog='greeter')
    parser.add_argument("name", nargs='?')
    parser.add_argument("--no-capitalize", action="store_const", const=True)

    args = parser.parse_args(args=args)

    print(greet(args.name, no_capitalize=args.no_capitalize))

    $ python greet.py dave --no-capitalize

In [7]:
greetcli(['dave', '--no-capitalize'])

Hello dave!


### What did ``greetcli`` do?

Positional argument ➡ Positional argument

Named argument ➡ Named argument

This is redundant!

* We create a parser, add arguments, parse, then call ``greet``
* Redundant when it's informations that's already there
* Python makes the signature available
* Let's use that

## Meet Clize

In [8]:
from clize import run

run(greet)

/home/epsy/.local/lib/python3.4/site-packages/IPython/kernel/__main__.py: Unknown option '-f'
Usage: /home/epsy/.local/lib/python3.4/site-packages/IPython/kernel/__main__.py [OPTIONS] [name]


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


In [9]:
from clize import run as orig_run

def rrun(*args, **kwargs):
    orig_run(*args, exit=False, **kwargs)

def run(*args, **kwargs): pass

    $ python greeter.py

In [10]:
rrun(greet, args=['greeter.py'])

Hello world!


    $ python greeter.py john

In [11]:
rrun(greet, args=['greeter.py', 'john'])

Hello John!


    $ python greeter.py dave --no-capitalize

In [12]:
rrun(greet, args=['greeter.py', 'dave', '--no-capitalize'])

Hello dave!


* Just one function call and the translation is done

In [13]:
def greet(name=None, *, no_capitalize=False):
    """Returns a greeting for a given person or the whole world.
    
    name: The name of the person to greet
    
    no_capitalize: Don't capitalize the given name
    """
    if name is None:
        return "Hello world!"
    else:
        if not no_capitalize:
            name = name.title()
        return "Hello {}!".format(name)

    $ python greet.py --help

In [14]:
rrun(greet, args=['greet.py', '--help'])

Usage: greet.py [OPTIONS] [name]

Returns a greeting for a given person or the whole world.

Arguments:
  name              The name of the person to greet

Options:
  --no-capitalize   Don't capitalize the given name

Other actions:
  -h, --help        Show the help


* Add param info in docstring -> nice --help

Next: Couple simple things.

## More simple things

### Want an int? ➞ Let the default be an int!

In [15]:
def greet(name, *, repeat=1):
    ... # the argument to repeat is always converted to an int

### Collect all positional ➞ Use ``*args``

In [16]:
def greet(*names):
    ... # name collects all positional arguments

### Want an alias for your option? ➞ Use an annotation

In [17]:
def greet(name=None, *, no_capitalize:'C'=False):
    ... # You can now use "python greet.py -C dave"

### Run multiple functions

In [18]:
def part(name):
    """Wishes someone goodbye.
    
    name: The person to address"""
    return "Goodbye {}".format(name)

In [19]:
run(greet, part)

* converts to default argument type
* parameters work just like in python, so ``*args`` works
* things like (short) parameter aliases don't exit in py, so annotation

Next: Because we deal with functions... neat stuff!

## Compositing functions
### Decorators

In [26]:
from sigtools.wrappers import wrapper_decorator

@wrapper_decorator
def noisy(wrapped, *args, quiet=False, **kwargs):
    """
    repeat: Amount of times the message is repeated
    """
    if not quiet: print("Running 'wrapped'")
    ret = wrapped(*args, **kwargs)
    if not quiet: print("'wrapped' finished")
    return ret

@noisy
def greet(*names: name_argument):
    ...

@noisy
def part(*names: name_argument):
    ...

* You can write decorators that work just like regular ones

### A mini-CLI for each parameter

In [30]:
from clize.parameters import argument_decorator

@argument_decorator
def name_argument(val, *, no_capitalize:'C'=False):
    """For each name:
    
    no_capitalize: Don't capitalize this name"""
    if not no_capitalize:
        return val.title()
    return val

def greet(*names: name_argument):
    ...

* You can add options individual to each occurence of a parameter

# ``pip install --user clize``

https://clize.readthedocs.org/

[Clize community on Google+](https://plus.google.com/communities/101146333300650079362)

http://epsy.github.io/clize/

### Yann Kaiser

[@YannKsr](https://twitter.com/YannKsr)

[kaiser.yann@gmail.com](mailto:kaiser.yann@gmail.com)

* Check RTD for tutorials
* need help for FAQ page
* will post slides link on meetup group